#### PyTorch CNN MNIST

In [1]:
import torch
import numpy as np

import idx2numpy
import matplotlib.cm as cm
import matplotlib.pyplot as plt

PATH = 'C:/Projects/keras_talk/keras/intern/0709/MNIST/'
EPOCHS = 20
BATCH_SIZE = 50
D=(60000//BATCH_SIZE) // 50

def one_hot_encoding( y, L ):
    Encoded_y = []
    N = len(y)
    for number in range(N):
        encoding = [0]*L
        np.asarray(encoding)
        encoding[ y[number] ] = 1
        Encoded_y.append(encoding)
    
    return np.asarray(Encoded_y)


#### data

In [2]:
x_test_path = 't10k-images.idx3-ubyte'
y_test_path = 't10k-labels.idx1-ubyte'
x_train_path = 'train-images.idx3-ubyte'
y_train_path = 'train-labels.idx1-ubyte'


x_test = idx2numpy.convert_from_file(PATH+x_test_path)
y_test = idx2numpy.convert_from_file(PATH+y_test_path)
x_train = idx2numpy.convert_from_file(PATH+x_train_path)
y_train = idx2numpy.convert_from_file(PATH+y_train_path)


x_test  = x_test.reshape ( len(x_test),    28* 28 ).astype('float32') / 255.
x_train = x_train.reshape( len(x_train),   28* 28 ).astype('float32') / 255.        
#y_test  = one_hot_encoding(y_test,10)
#y_train = one_hot_encoding(y_train,10)


#### Data_loader 구성

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


x_test  = torch.from_numpy(x_test ).float().to('cpu')
x_train = torch.from_numpy(x_train).float().to('cpu')
y_test  = torch.from_numpy(y_test ).long().to('cpu')
y_train = torch.from_numpy(y_train).long().to('cpu')


dataset      = TensorDataset(x_train, y_train)
dataset_test = TensorDataset(x_test, y_test)

dataloader      = DataLoader( dataset, batch_size = BATCH_SIZE
                        ,shuffle= True, drop_last = True ) 
dataloader_test = DataLoader( dataset_test, batch_size = BATCH_SIZE
                        ,shuffle= True, drop_last = True )

..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


#### pytorch model 구성

In [4]:
import torch.nn
import torch.nn.functional
import torch.optim

class MLP( torch.nn.Module ):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.lay1  = torch.nn.Linear( 28*28, 256 )
        self.relu1 = torch.nn.ReLU()
        self.lay2  = torch.nn.Linear( 256, 256 )
        self.relu2 = torch.nn.ReLU()
        self.lay3  = torch.nn.Linear( 256, 256 )
        self.relu3 = torch.nn.ReLU()
        self.lay4  = torch.nn.Linear( 256, 128 )
        self.relu4 = torch.nn.ReLU()
        self.lay5  = torch.nn.Linear( 128, 128 )
        self.relu5 = torch.nn.ReLU()
        self.lay6  = torch.nn.Linear( 128, 10 )
        self.relu6 = torch.nn.ReLU()
        self.dropout= torch.nn.Dropout()
        
        
    def forward(self, x):
        output = self.lay1(x)
        output = self.relu1(output)
        output = self.lay2(output)
        output = self.relu2(output)
        output = self.lay3(output)
        output = self.relu3(output)
        output = self.lay4(output)
        output = self.relu4(output)
        output = self.lay5(output)
        output = self.relu5(output)
        output = self.lay6(output)
        output = self.relu6(output)
        output = self.dropout(output)
        output = torch.nn.functional.softmax(output, dim=1)
        
        return output
        


#### model 학습

In [5]:

torch.manual_seed(1)

model = MLP()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


model.train()
for epoch in range(EPOCHS):
    train_loss = 0.0
    
    print('epoch'+str(epoch+1)+':  [', end='')
    for i,(data, label) in enumerate(dataloader):

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        total = label.size(0)
        preds = torch.max(output.data,1)[1]
        correct = (preds == label).sum().item()
        
        
  
        #progress bar
        if D == i:
            print('%', end='')
            D += D
            
    print('%]')
    D = (60000//BATCH_SIZE)//50        
        
    #summary 
    print('    loss: ', loss.item(), end='\t')
    print('acc:  ', 100.*correct/total)




epoch1:  [%%%%%%%]
    loss:  2.0973684787750244	acc:   30.0
epoch2:  [%%%%%%%]
    loss:  2.06443190574646	acc:   32.0
epoch3:  [%%%%%%%]
    loss:  1.9467321634292603	acc:   52.0
epoch4:  [%%%%%%%]
    loss:  2.034698724746704	acc:   36.0
epoch5:  [%%%%%%%]
    loss:  2.019996404647827	acc:   38.0
epoch6:  [%%%%%%%]
    loss:  2.107069730758667	acc:   28.0
epoch7:  [%%%%%%%]
    loss:  1.9865912199020386	acc:   44.0
epoch8:  [%%%%%%%]
    loss:  2.071624994277954	acc:   30.0
epoch9:  [%%%%%%%]
    loss:  2.004092216491699	acc:   40.0
epoch10:  [%%%%%%%]
    loss:  2.039726972579956	acc:   38.0
epoch11:  [%%%%%%%]
    loss:  1.9833678007125854	acc:   38.0
epoch12:  [%%%%%%%]
    loss:  2.0476770401000977	acc:   38.0
epoch13:  [%%%%%%%]
    loss:  2.000324010848999	acc:   38.0
epoch14:  [%%%%%%%]
    loss:  2.0387167930603027	acc:   38.0
epoch15:  [%%%%%%%]
    loss:  1.9508213996887207	acc:   48.0
epoch16:  [%%%%%%%]
    loss:  2.0368645191192627	acc:   38.0
epoch17:  [%%%%%%%]
    lo

#### 학습 결과

In [6]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for data, label in dataloader_test:
        output = model(data)
        preds  = torch.max(output.data, 1)[1]
        total   += len(label)
        correct += (preds==label).sum().item()
    print('Test Accuracy: ', 100.*correct/total)

Test Accuracy:  68.76
